<a href="https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/BioEmu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Biomolecular Emulator (BioEmu) in ColabFold**
<img src="https://github.com/microsoft/bioemu/raw/main/assets/emu.png" height="130" align="right" style="height:240px">

[BioEmu](https://github.com/microsoft/bioemu) is a framework for emulating biomolecular dynamics and integrating structural prediction tools to accelerate research in structural biology and protein engineering. This notebook builds uses ColabFold to generate the MSA.



For more details, please read the [BioEmu Preprint](https://www.biorxiv.org/content/10.1101/2024.12.05.626885v1.abstract).


In [ ]:
#@title **Enter Your Protein Sequence & Sampling Parameters**
sequence = "GYDPETGTWG"  #@param {type:"string"}
num_samples = 10  #@param {type:"integer"}
output_dir = "/content/test-chignolin"  #@param {type:"string"}
side_chain_reconstruction = True  #@param {type:"boolean"}

print(f"Sequence: {sequence}\nNumber of samples: {num_samples}\nOutput directory: {output_dir}")

In [ ]:
#@title Install
%%time
import sys
import os


print("Cloning BioEmu...")
!git clone -q https://github.com/microsoft/bioemu.git

if not os.path.isfile("COLABFOLD_READY"):
  print("installing colabfold...")
  os.system("pip install -q --no-warn-conflicts 'colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold'")
  if os.environ.get('TPU_NAME', False) != False:
    os.system("pip uninstall -y jax jaxlib")
    os.system("pip install --no-warn-conflicts --upgrade dm-haiku==0.0.10 'jax[cuda12_pip]'==0.3.25 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html")
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabfold colabfold")
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/alphafold alphafold")
  os.system("touch COLABFOLD_READY")

print("Patching colabfold installation...")
os.system("patch /usr/local/lib/python3.*/dist-packages/alphafold/model/modules.py bioemu/colabfold_setup/modules.patch")

patch_text = r"""
430d429
<
478a478
>             np.save(files.get("single_repr_evo", "npy"), result["representations_evo"]["single"])
480a481
>             np.save(files.get("pair_repr_evo", "npy"), result["representations_evo"]["pair"])
"""

with open("inline_batch.patch","w") as f:
    f.write(patch_text)

os.system("patch /usr/local/lib/python3.*/dist-packages/colabfold/batch.py inline_batch.patch")

# Optionally install side-chain reconstruction tools (HPacker)
if side_chain_reconstruction:
  print("Enabling side-chain reconstruction tools (HPacker)...")
  if not os.path.exists("hpacker"):
      !git clone -q https://github.com/gvisani/hpacker.git
      %cd hpacker
      !pip install -q --no-warn-conflicts .
      %cd ..
  else:
      print("HPacker repo already cloned.")
else:
  print("Side-chain reconstruction not selected. Skipping HPacker installation.")

# Finally, install the BioEmu package itself.
if not os.path.isfile("BIOEMU_READY"):
  if not os.path.exists("bioemu"):
      %cd bioemu
  else:
      %cd bioemu
  !pip install -q --no-warn-conflicts ".[md]"
  os.system("touch BIOEMU_READY")

print("Installation complete.")

In [ ]:
#@title Run BioEmu Sampling
import subprocess, sys

cmd = [
    sys.executable, "-m", "bioemu.sample",
    "--sequence", sequence,
    "--num_samples", str(num_samples),
    "--output_dir", output_dir
]

print("Running command:\n", " ".join(cmd))
subprocess.run(cmd)
print("\nSampling complete. Check the", output_dir, "directory for PDB and XTC files.")

In [ ]:
#@title Visualize Frames

import os
import tempfile
import ipywidgets as widgets
from IPython.display import display
import py3Dmol
import mdtraj as md

# Provide your actual output_dir, or ensure these files exist
pdb_path = os.path.join(output_dir, "topology.pdb")
xtc_path = os.path.join(output_dir, "samples.xtc")
def frame_to_pdb_string(frame):
    """Write a single-frame mdtraj.Trajectory to a temp PDB file, return as string."""
    with tempfile.NamedTemporaryFile(mode='w', suffix='.pdb', delete=False) as tmp:
        tmp_filename = tmp.name
    frame.save_pdb(tmp_filename)
    with open(tmp_filename, 'r') as f:
        pdb_str = f.read()
    os.remove(tmp_filename)
    return pdb_str

# Create the py3Dmol view once
view = py3Dmol.view(width=600, height=400)
traj = md.load(xtc_path, top=pdb_path)
pdb_str = frame_to_pdb_string(traj[:1])
view.addModel(pdb_str, "pdb")
view.setStyle({"cartoon": {"color": "spectrum"}})
view.zoomTo()
view.show()  # Show the viewer just once here

# Create the slider
frame_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(traj) - 1,
    step=1,
    description='Frame'
)

def on_frame_change(change):
    """Update the existing py3Dmol view in place when the slider changes."""
    frame_idx = change["new"]
    pdb_str = frame_to_pdb_string(traj[frame_idx:frame_idx+1])

    view.removeAllModels()
    view.addModel(pdb_str, "pdb")
    view.setStyle({"cartoon": {"color": "spectrum"}})
    view.zoomTo()
    # Force an update of the existing view instead of creating a new cell output
    view.update()

frame_slider.observe(on_frame_change, names="value")
display(frame_slider)


In [ ]:
#@title Run Side-Chain Reconstruction
import subprocess, sys
if side_chain_reconstruction:
  pdb_path = os.path.join(output_dir, "topology.pdb")
  xtc_path = os.path.join(output_dir, "samples.xtc")

  if not (os.path.isfile(pdb_path) and os.path.isfile(xtc_path)):
      print("Could not find PDB/XTC for side-chain reconstruction.")
  else:
      cmd_sidechain = [
          sys.executable, "-m", "bioemu.sidechain_relax",
          "--pdb-path", pdb_path,
          "--xtc-path", xtc_path,
          "--no-md-equil"  # remove this arg if you want a short MD equilibration
      ]
      print("Running side-chain reconstruction:\n", " ".join(cmd_sidechain))
      subprocess.run(cmd_sidechain)
      print("\nSide-chain reconstruction complete! Check 'samples_sidechain_rec.pdb/xtc'.")

In [ ]:
#@title Zip and Download All Results
from google.colab import files

zip_name = "bioemu_samples.zip"
!zip -r $zip_name $output_dir
files.download(zip_name)